# Classification MNSIT Datasets
- 로컬 컴퓨터를 이용할 때는 에폭을 낮게 지정(단지 실행이 되는지만 확인)
- Kaggle이나 gcp를 이용할 때는 GPU를 사용하여 실제 훈련 실시

##  라이브러리 import

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Datasets 다운로드 및 불러오기

In [2]:
# Dataset & DataLoader

transform = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Normalize([0], [1])
])

trainset = torchvision.datasets.MNIST(root = './data', train = True,
                                      download = True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 128,
                                          shuffle = True, num_workers=1)

testset = torchvision.datasets.MNIST(root = './data', train = False,
                                     download = True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 128,
                                         shuffle = True, num_workers=1)

classes = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

print(trainset)
print('*'*100)
print(testset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0], std=[1])
           )
****************************************************************************************************
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0], std=[1])
           )


## Datasets 확인하기

In [ ]:
# Visualization Datasets

def show(img):
    print(img.size())
    grid = torchvision.utils.make_grid(img, padding = 0) # make_grid 함수는 3채널로 만든다(모두 같은 format으로)
    print(grid.size())
    tranimg = grid.permute(1,2,0)
    print(tranimg.size())
    plt.imshow(tranimg, aspect = 'auto')

images, labels = iter(trainloader).next()
show(images)

# Model 구축하기

In [ ]:
# Make Model

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 7, padding = 3)
        self.conv2 = nn.Conv2d(10, 50, 7, padding = 3)
        self.conv3 = nn.Conv2d(50, 120, 7, padding = 3)
        self.conv4 = nn.Conv2d(120, 100, 5)
        self.conv5 = nn.Conv2d(100, 20, 5)
        self.conv6 = nn.Conv2d(20, 10, 5)
        self.conv7 = nn.Conv2d(10, 10, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(10 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 360)
        self.fc3 = nn.Linear(360, 50)
        self.fc4 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x)) # 28 28
        x = F.relu(self.conv2(x)) # 28 28
        x = F.relu(self.conv3(x)) # 28 28
        x = F.relu(self.conv4(x)) # 24 24
        x = F.relu(self.conv5(x)) # 20 20
        x = self.pool(F.relu(self.conv6(x))) # 8 8
        x = x.view(-1, 10 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

## GPU 사용 여부 판단하기

In [ ]:
# GPU 사용여부 판단
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('We can use GPU')
else:
    device = torch.device('cpu')
    print('We can use CPU')

## Model 및 Optimizer 생성
- Model Parameters 확인

In [ ]:
import torch.optim as optim

model = Net().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)
print(loss_func, optimizer)

In [ ]:
for p in model.parameters():
    print(p.size())

## 모델 작동상태 확인하기

In [ ]:
model.eval()
with torch.no_grad():
    images, labels = next(iter(trainloader))
    images, labels = images.to(device), labels.to(device)
    print(images.size())
    example = model(images)
    print('Test : ', example)

## 학습하기

In [ ]:
# Train

import time

EPOCH = 1

for e in range(1, EPOCH+1):
    model.train()
    start_time = time.time()
    running_loss = 0
    
    for i, data in enumerate(trainloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        now = time.time()
        print('\r[%d/%d]-----[%d/%d] LOSS : %.3f------ Time : %d' 
              %(e, EPOCH, i, 60000/128, running_loss, now - start_time), end = '')
        
    print('\n')

## Model 저장하기

In [ ]:
torch.save(model, 'data/mnist_classifier.pth')
test_model = torch.load('data/mnist_classifier.pth')
test_model

# 예측이 작동하는지 확인하기

In [ ]:
with torch.no_grad():
    classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
    testloader = torch.utils.data.DataLoader(testset, batch_size = 8,
                                         shuffle = True, num_workers=1)
    test_data = iter(testloader)
    test_images, test_labels = test_data.next()
    show(test_images)
    test_images, test_labels = test_images.cuda(), test_labels.cuda()
    
    test_outputs = test_model(test_images)
    _, predicted = torch.max(test_outputs, 1)
    
    
    print('GroundTruth : ', ' '.join(classes[test_labels[j]] for j in range(8)))
    print('Predicted : ', ' '.join(classes[predicted[i]] for i in range(8)))


## Model 성능 시험하기

In [ ]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for data in testloader:
        val_images, val_labels = data
        val_images, val_labels = val_images.to(device), val_labels.to(device)
        
        val_outputs = model(val_images)
        #_, val_predicted = torch.max(val_outputs.data, 1)  # 이것 보다는
        pred = val_outputs.argmax(dim=1, keepdim=True)
        #correct += (val_predicted == val_labels).sum().item() # 이것 보다는
        correct += pred
print('Accuracy of the network on the 10000 test images : %.3f %%' %(100 * correct / total))

In [ ]:
isinstance(Net(), nn.Conv2d)

In [ ]:
type(nn.Conv2d)

In [ ]:
type(nn.Conv2d(1, 10, 7)) == nn.Conv2d

In [ ]:
type(model.conv1) == nn.Conv2d

In [ ]:
self.conv1 = nn.Conv2d(1, 10, 7, padding = 3)

In [ ]:
a = 10
if a == (9 or 10):
    print('g')

In [ ]:
def weight_init(m):
    if ((type(m) == nn.Conv2d) or (type(m) == nn.Linear)):
        print('0')
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0)
        
model.apply(weight_init)

In [ ]:
nn.init.xavier_normal(torch.zeros(3,5))